# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Import etl_helper.py

In [2]:
from etl_helper import *

#### Creating list of filepaths to process original event csv data files

In [3]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/rvdubey/workspace/data_eng_nanodegree/data_modeling_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
data_filename = 'event_datafile_new.csv'

In [5]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open(data_filename, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
with open(data_filename, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [8]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}""")

#### Set Keyspace

In [9]:
session.set_keyspace('sparkify')

### Now we will create tables to answer the following queries. The query helps us determine how we should go about and create our tables, and design our schema

#### Q1: Give me the artist, song title, and song's length in the music app event history that was during sessionId=338 and itemInSession=4 


Note here that we need to perform a `WHERE` clause on `session` and `itemInSession`. Moreover, we see that these two columns uniquely identify a row in our events data.

So we can go ahead and create `sessionized_music_table` that partitions data by `(sessionId, itemInSession)` 

NOTE: the `COMPOSITE PRIMARY KEY` are the first two columns in the `CREATE` statement

In [10]:
create_table_query = """
CREATE TABLE IF NOT EXISTS
sessionized_music_table
    (
    sessionId int, 
    itemInSession int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (sessionId, itemInSession))"""
session.execute(create_table_query)

Now let's insert data into the above table using the etl_helper functions. 

**Note:** We need to make sure we insert values such that the primary-keys are present in same order as they are in the `COMPOSITE PRIMARY KEY` as shown in the `CREATE` statement. Thus, `sessionId` and `itemInSession` need to be the first two values we insert via our insert statements.

In [11]:
insert_data_in_table(
    session, 
    data_filename, 
    'sessionized_music_table', 
    ('sessionId', 'itemInSession', 'artist', 'song', 'length')
)

Inserting data into sessionized_music_table...
Finished inserting data into sessionized_music_table


**Finally, we can run our query:**

`Give me the artist, song title, and song's length in the music app event history that was during sessionId=338 and itemInSession=4`

NOTE: The order of columns in the `WHERE` clause matches that of the `COMPOSITE PRIMARY KEY`

In [12]:
result = session.execute("""
SELECT artist, song, length FROM sessionized_music_table where sessionId=338 AND itemInSession=4""")

print (f"""\nResults:""")
for r in result:
    print(r)  


Results:
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


#### Q1: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Note here that we need to perform a `WHERE` clause on `userId` and `sessionid`.  So we definitely need these two columns in our `COMPOSITE PRIMARY KEY`. But, we can see that this does not uniquely identify a row in our events data, since a particular userId in a particular sessionId can have multiple itemInSession. 

So, we need to add `itemInSession` to our `COMPOSITE PRIMARY KEY` to uniquely identify a row. Moreover, we need to return results such that they are sorted by `itemInSession` so we need to make this our `CLUSTERING COLUMN`. We can choose to sort this be ASC or DESC order.

So we can go ahead and create `user_sessionized_music_table` that partitions data by `(userId, sessionId)` and sorts them by `itemInSession`.  
Thus, our `PRIMARY KEY` becomes `((userId, sessionId), itemInSession)`

NOTE: the `COMPOSITE PRIMARY KEY` are the first three columns in the `CREATE` statement

In [13]:
create_table_query = """
CREATE TABLE IF NOT EXISTS
user_sessionized_music_table
    (
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY ((userId, sessionId), itemInSession)
    )
    WITH CLUSTERING ORDER BY (itemInSession DESC);"""
session.execute(create_table_query)

Again, let's insert data into the above table using the etl_helper functions. 

**Note:** We need to make sure we insert values such that the primary-keys are present in same order as they are in the `COMPOSITE PRIMARY KEY` as shown in the `CREATE` statement. Thus, `userId`, `sessionId`, and `itemInSession` need to be the first three values **(in the same order)** we insert via our insert statements.

In [14]:
insert_data_in_table(
    session, 
    data_filename, 
    'user_sessionized_music_table', 
    ('userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName')
)

Inserting data into user_sessionized_music_table...
Finished inserting data into user_sessionized_music_table


**Finally, we can run our query:**

`Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182`

NOTE: The order of columns in the `WHERE` clause matches that of the `COMPOSITE PRIMARY KEY`

In [15]:
print (f"""\nResults:""")
result = session.execute("""
SELECT artist, song, firstName, lastName, itemInSession FROM user_sessionized_music_table where userId=10 AND sessionId=182""")
for r in result:
    print(r)


Results:
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)


#### Q3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Note here that we need to perform a `WHERE` clause on `song`.  So we definitely need this columns in our `PRIMARY KEY`. But, we know that we cannot uniquely identify a row just by the `song` so we can pair it with `userId` (*since this query **only** cares about the users who listen to a particular song, and not about the frequency with which they listen to a song, if we cared about frequency then we would have to include **sessionId** and **itemInSession** too*)

So, we need to add `song` and `userId` in our `COMPOSITE PRIMARY KEY` to uniquely identify a row. But we need to make sure that `song` is the first column in our `COMPOSITE PRIMARY KEY` since we are querying by it. 

So we can go ahead and create `user_songs_table` that partitions data by `(song, userId)`

NOTE: the `COMPOSITE PRIMARY KEY` are the first two columns in the `CREATE` statement

In [16]:
create_table_query = """
CREATE TABLE IF NOT EXISTS
user_songs_table
    (
    song text,
    userId int,
    firstName text,
    lastName text,
    PRIMARY KEY (song, userId)
    );"""
session.execute(create_table_query)

Let's insert data into the above table using the etl_helper functions. 

**Note:** We need to make sure we insert values such that the primary-keys are present in same order as they are in the `COMPOSITE PRIMARY KEY` as shown in the `CREATE` statement. Thus, `song` and `userId` need to be the first two values **(in the same order)** we insert via our insert statements.

In [17]:
insert_data_in_table(
    session,
    data_filename, 
    'user_songs_table', 
    ('song', 'userId','firstName', 'lastName')
)

Inserting data into user_songs_table...
Finished inserting data into user_songs_table


**We can run our query: ** 

`Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'`

NOTE: The order of columns in the `WHERE` clause matches that of the `COMPOSITE PRIMARY KEY`

In [18]:
print (f"""\nResults:""")
result = session.execute("""
SELECT  firstName, lastName  FROM user_songs_table where song='All Hands Against His Own'""")
for r in result:
    print(r)


Results:
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [19]:
table_list = session.execute("""SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'sparkify';""")
for t in table_list:
    print(t.table_name)
    session.execute(f"DROP TABLE IF EXISTS {t.table_name}")

sessionized_music_table
user_sessionized_music_table
user_songs_table


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()